In [58]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier

from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import string
from typing import Counter
from NBClassifier import NBClassifier

In [86]:
df1 = pd.read_csv('SMSSpamCollection',sep='\t',header=None)
df2 = pd.read_csv('mmm.csv')

df1.columns = ['Label', 'Mail']
df1['Label'] = df1['Label'].replace(['spam'],1)
df1['Label'] = df1['Label'].replace(['ham'],0)
df2.columns = ['Label', 'Mail']
df = pd.concat([df1,df2], axis=0)

print(df.shape)
df.head()

(10743, 2)


,Label,Mail
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [87]:
df.drop_duplicates(inplace = True)
df.shape

(10162, 2)

In [40]:
def process_text(text):
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    
    clean_words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    
    return clean_words

In [5]:
df['Mail'] = df['Mail'].apply(process_text)
df['Mail']

0       [Subject, enron, methanol, meter, 988291, foll...
1       [Subject, hpl, nom, january, 9, 2001, see, att...
2       [Subject, neon, retreat, ho, ho, ho, around, w...
3       [Subject, photoshop, windows, office, cheap, m...
4       [Subject, indian, springs, deal, book, teco, p...
                              ...                        
5165    [Subject, fw, crosstex, energy, driscoll, ranc...
5166    [Subject, put, 10, ft, transport, volumes, dec...
5167    [Subject, 3, 4, 2000, following, noms, hpl, ta...
5169    [Subject, industrial, worksheets, august, 2000...
5170    [Subject, important, online, banking, alert, d...
Name: Mail, Length: 4993, dtype: object

In [88]:
randomized = df.sample(frac=1)
train_size = round(len(randomized) * 0.8)
train = randomized[:train_size].reset_index(drop=True)
test = randomized[train_size:].reset_index(drop=True)
print(train.shape)
print(test.shape)

(8130, 2)
(2032, 2)


In [7]:
bag_word = []
testmails= test['Mail']
test_labels = test['Label']
testmails2 = []
trainmails = train['Mail']
train_labels = train['Label']
trainmails2= []

In [8]:
for doc in trainmails:
    words = doc
    bag_word += words

bag_word = set(bag_word)
bag_word = list(bag_word)

def tovector(mail):   
    splitmail = mail
    mvec = np.zeros(len(bag_word))
    count = Counter(splitmail)
    for i, word in enumerate(bag_word):
                mvec[i]+=count[word]
    
    return mvec

In [9]:
for i,mail in enumerate(testmails):

    testmails2.append(tovector(mail))

for i,mail in enumerate(trainmails):

    trainmails2.append(tovector(mail))

In [10]:
d = {'Mail': trainmails2, 'Label': train_labels}
Trainingdata = pd.DataFrame(data=d)
d = {'Mail': testmails2, 'Label': test_labels}
Testdata = pd.DataFrame(data=d)

print(Testdata)

                                                  Mail  Label
0    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      0
1    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      0
2    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      0
3    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      0
4    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      1
..                                                 ...    ...
994  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      0
995  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      1
996  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      0
997  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      0
998  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      0

[999 rows x 2 columns]


In [11]:
model= NBClassifier()
model.fit(Trainingdata['Mail'],Trainingdata['Label'])
result = model.predict(Testdata)

model.score(Testdata)

0.9819819819819819

In [89]:
pipe = Pipeline(steps=[('vectorize', CountVectorizer(ngram_range=(1, 1), token_pattern=r'\b\w+\b')),
                       ('classifier', MultinomialNB() )])

pipe.fit(train['Mail'], train['Label'])

Pipeline(steps=[('vectorize', CountVectorizer(token_pattern='\\b\\w+\\b')),
                ('classifier', MultinomialNB())])

In [90]:
y_predict = pipe.predict(test['Mail'])

print(accuracy_score(test['Label'], y_predict))

0.9542322834645669


In [91]:
test['predict'] = y_predict
test

,Label,Mail,predict
0,0,"Subject: re : deer park railroad crossing 18 ""...",0
1,1,Latest Nokia Mobile or iPOD MP3 Player +£400 p...,1
2,0,Then why no one talking to me,0
3,1,"As a SIM subscriber, you are selected to recei...",1
4,0,Subject: jan . 01 sale to texas general land o...,0
...,...,...,...
2027,0,Subject: training resources and cost to attend...,0
2028,1,Subject: cheating house wife services\r\ndisco...,1
2029,0,Subject: change in scada server to nahou - sca...,0
2030,0,"Subject: re : exxon company , usa global # 960...",0


In [92]:
accuracy = sum(test.Label == test.predict)
accuracy

1939

In [24]:
pipe = Pipeline(steps=[('vectorize', CountVectorizer(ngram_range=(1, 1), token_pattern=r'\b\w+\b')),
                       ('classifier', LogisticRegression())])

pipe.fit(train['Mail'], train['Label'])

y_predict = pipe.predict(test['Mail'])

print(accuracy_score(test['Label'], y_predict))


0.978978978978979


In [25]:
pipe = Pipeline(steps=[('vectorize', CountVectorizer(ngram_range=(1, 1), token_pattern=r'\b\w+\b')),
                       ('classifier', SVC(kernel = 'linear'))])

pipe.fit(train['Mail'], train['Label'])

y_predict = pipe.predict(test['Mail'])

print(accuracy_score(test['Label'], y_predict))

0.9679679679679679


In [26]:
pipe = Pipeline(steps=[('vectorize', CountVectorizer(ngram_range=(1, 1), token_pattern=r'\b\w+\b')),
                       ('classifier', RandomForestClassifier(n_estimators=100))])

pipe.fit(train['Mail'], train['Label'])

y_predict = pipe.predict(test['Mail'])

print(accuracy_score(test['Label'], y_predict))

0.972972972972973


In [27]:
pipe = Pipeline(steps=[('vectorize', CountVectorizer(ngram_range=(1, 1), token_pattern=r'\b\w+\b')),
                       ('classifier', MLPClassifier())])

pipe.fit(train['Mail'], train['Label'])

y_predict = pipe.predict(test['Mail'])

print(accuracy_score(test['Label'], y_predict))

0.985985985985986
